In [85]:
from sympy import * 
import numpy as np
from scipy.stats import norm, multivariate_normal


# Topic 1

In [86]:
x = np.array([1, 1.5, 1.2, 5.8, 6.2, 6.5]).reshape(-1, 1)
mu_0 = [2, 3]
sd_0 = [0.1,0.1] 
pi_0 = [0.5,0.5] 

In [87]:
gamma = lambda x, k, mus, sds, pis : pis[k] * norm.pdf(x, loc = mus[k], scale = sds[k])/(pis[0] * norm.pdf(x, loc = mus[0], scale = sds[0]) + pis[1] * norm.pdf(x, loc = mus[1], scale = sds[1]))
muk = lambda Nk, gammak, x : (1/Nk) * np.dot(gammak.T, x) 
sdk = lambda Nk, gammak, x, muk : np.sum([gammak * (xn - muk) * (xn - muk) for xn in x])
pik = lambda Nk, N : Nk/N
N1 = [np.sum(gamma(x, 0, mu_0, sd_0, pi_0)), np.sum(gamma(x, 0, mu_0, sd_0, pi_0))] 
gamma1 = [gamma(x, 0, mu_0, sd_0, pi_0), gamma(x, 1, mu_0, sd_0, pi_0)]
mu1 = [muk(N1[0],gamma1[0], x), muk(N1[1],gamma1[1], x)]
sd1 = [sdk(N1[0], gamma1[0], x, mu1[0]), sdk(N1[0], gamma1[0], x, mu1[0])]
pi1 = [pik(N1[0], len(x)), pik(N1[1], len(x))]

print(f"{N1=}\n{gamma1=}\n{mu1=}\n{sd1=}\n{pi1=}")

N1=[np.float64(3.0), np.float64(3.0)]
gamma1=[array([[1.00000000e+000],
       [1.00000000e+000],
       [1.00000000e+000],
       [4.81749166e-144],
       [0.00000000e+000],
       [0.00000000e+000]]), array([[7.17509597e-66],
       [3.72007598e-44],
       [3.48110684e-57],
       [1.00000000e+00],
       [1.00000000e+00],
       [1.00000000e+00]])]
mu1=[array([[1.23333333]]), array([[6.16666667]])]
sd1=[np.float64(220.16), np.float64(220.16)]
pi1=[np.float64(0.5), np.float64(0.5)]


# Topic 2 

## Code I've previously used for expectation maximization of multivariate normals

In [88]:
def gaussian_mixture(x, means, covariances, pis):
    pdfx = np.sum([pis[k] * multivariate_normal.pdf(x = x, mean = means[k], cov = covariances[k]) for k in range(len(means))], axis=0)
    return pdfx

In [89]:
def expectation_maximization_gaussian(mus_0, covariances_0, pis_0, data_x, on_step):
    # Evaluation
    # So the evaluation acts as an average of the k'th object with xn over all the K objects. 
    # The denorminator is exactly what I just did in previous function. 
    # The length K do I determine from the length of means.
    # The length of N do I determine from the length of datapoints x. 
    
    N, D = data_x.shape # Datapoints and dimension of x
    K = len(mus_0)
    gamma = np.zeros((N, K))
    for n in range(N): 
        denominator = gaussian_mixture(data_x[n], mus_0, covariances_0, pis_0)
        xn = data_x[n]
        
        # Gamma(znk)
        for k in range(K): 
            gamma[n, k] = (pis_0[k]/denominator) * multivariate_normal.pdf(x = xn, mean=mus_0[k], cov=covariances_0[k])
        
            
    # Now for reestimating     
    mean_new = np.zeros(mus_0.shape)
    cov_new = np.zeros(covariances_0.shape)
    pis_new = np.zeros(pis_0.shape)
    
    # Functions: 
    meank = lambda Nk, gamma, x : (1/Nk) * np.sum(gamma[:, k].reshape(-1, 1) * x[:], axis = 0)
    covk = lambda Nk, gamma, xn, mean_new : (1/Nk) * gamma[n, k] * np.outer(xn - mean_new,(xn - mean_new).T)       
    for k in range(K): 
        Nk = np.sum(gamma[:, k]) # Nk = np.sum(n=1 -> N (gammaznk in R^150x3))    => sum = [sum_k1, sum_k2, sum_k3] = np.sum(gamma[:, k]), iterating over every k
        mean_new[k] = meank(Nk, gamma, data_x)
        for n in range(N): 
            cov_new[k] += covk(Nk, gamma, data_x[n], mean_new[k])     
        pis_new[k] = Nk/N
      
        
    # Evaluation: 
    # The logLikeliHood is just the sum over the natural logarithm to the gaussian mixture. It's the exact same as the denominator in gamma. 
    logLikeliHood = lambda pis, x, mean, cov : np.sum([
        np.log(
            np.sum([pis[k] * multivariate_normal.pdf(x[n], mean[k], cov[k]) 
                    for k in range(K)])) 
        for n in range(N)])
    
    # I haven't been given a exact criteria on when to stop, so make me do it when |logLikeliHoodk - logLikeliHoodk-1| < 0.00001
    criteria = 0.00001
    logLikeliHoodk_1 = logLikeliHood(pis =   pis_0, x = data_x, mean =    mus_0, cov = covariances_0)
    logLikeliHoodk =   logLikeliHood(pis = pis_new, x = data_x, mean = mean_new, cov =       cov_new)
    
    # Appending data before checking criteria. 
    predictions = np.argmax(gamma, axis=1) # gamma in R^150x3, for every n which of the columns is highest? The column index == k therefore this gives me k. 
    on_step(mean_new, cov_new, pis_new, logLikeliHoodk, predictions)
    
    if np.abs(logLikeliHoodk - logLikeliHoodk_1) < criteria: 
        return
    else : 
        expectation_maximization_gaussian(mus_0=mean_new, covariances_0=cov_new, pis_0=pis_new, data_x=data_x, on_step=on_step)     


## Trying to solve for the values.

In [93]:
np.random.seed(26)

classes = 3

all_steps_em = []
x = np.array([[[2,3],
               [4,3],
               [4,5]], 
              [[6,6],
               [7,5],
               [7,7]]])
mus_0 = x[0]
covariances_0 = np.array([np.cov(x[0].T)] * classes)
pis_0 = np.array([1/classes] * classes)

expectation_maximization_gaussian(
    mus_0, covariances_0, pis_0, x[0],
    lambda mus, covs, pis, log_likelihood, predictions: all_steps_em.append((mus, covs, pis, log_likelihood, predictions))
)
print(len(all_steps_em))
 #print(all_steps_em[0])
# print("\n\n\n\n\n")
#print(all_steps_em[17])

LinAlgError: When `allow_singular is False`, the input matrix must be symmetric positive definite.

# Topic 4 

In [91]:
w1 = np.array([[ 0.6,  0.2, -0.1,  0.4],
               [ 0.7, -1.0,  0.3,  0.2],
               [-0.5, -0.3, -0.4, -0.8]])
w2 = np.array([0.7, 0.7, 0.6, -0.3, -0.7]).reshape(-1, 1)

## Given x1, what's y?

In [92]:
x1 = np.array([1, 0.1, -0.1]).reshape(-1, 1)
z = np.dot(x1.T, w1)[0]
pprint(f"{z=}")
z1 = np.hstack([1, z]).reshape(-1, 1)
y = np.dot(z1.T, w2)[0][0]
print(f"{y=}")

z=array([ 0.72,  0.13, -0.03,  0.5 ])
y=np.float64(0.941)
